In [1]:
import pandas as pd

C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12336\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import

In [23]:
df_post=pd.read_excel('D:\\MIT PGDAML\\Neural Network\\Project\\Filtered_News_ET - NextDay.xlsx')

In [21]:
df_post

,News,filtered,Date,Entity,Symbol,Open,Close,Perc_Change,Sentiment
0,"Technical Breakout Stocks: How to trade HEG, 3...","Technical Breakout Stocks: How to trade HEG, 3...",2023-12-27,3M India,3MINDIA,31127.000000,31447.599609,1.029973,NaN
1,"Aarti Industries, HDFC AMC among 11 stocks tha...","Aarti Industries, HDFC AMC among 11 stocks tha...",2020-07-01,Aarti Industries,AARTIIND,466.500000,465.700012,-0.171487,NaN
2,"Hot Stocks: Brokerage views on Wipro, Federal ...","Hot Stocks: Brokerage views on Wipro, Federal ...",2023-07-14,Aarti Industries,AARTIIND,456.100006,448.299988,-1.710155,NaN
3,"Hot Stocks: Brokerages on Tata Communication, ...","Hot Stocks: Brokerages on Tata Communication, ...",2023-07-21,ABB India,ABB,4500.000000,4204.000000,-6.577778,NaN
4,"HCL Tech, ABB India among 6 stocks to trade ex...","HCL Tech, ABB India among 6 stocks to trade ex...",2023-04-23,ABB India,ABB,3164.699951,3217.399902,1.665243,NaN
...,...,...,...,...,...,...,...,...,...
13165,"Zydus Lifesciences, Poly Medicure among 10 sto...","Zydus Lifesciences, Poly Medicure among 10 sto...",2023-09-14,Zydus Lifesciences,ZYDUSLIFE,625.150024,643.250000,2.895301,NaN
13166,"Technical breakout: SBI, Zydus Wellness, 5 oth...","Technical breakout: SBI, Zydus Wellness, 5 oth...",2023-08-09,Zydus Wellness,ZYDUSWELL,1482.050049,1515.750000,2.273874,NaN
13167,Stock Radar: Bullish crossover on RSI makes Zy...,Stock Radar: Bullish crossover on RSI makes Zy...,2022-10-25,Zydus Wellness,ZYDUSWELL,1729.000000,1734.300049,0.306538,NaN
13168,Zydus Wellness promoters' stake hike point to ...,Zydus Wellness promoters' stake hike point to ...,2023-03-31,Zydus Wellness,ZYDUSWELL,1560.349976,1554.500000,-0.374914,NaN


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from transformers import pipeline

In [10]:
model=pipeline('sentiment-analysis', model='mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [24]:
df_post['filtered'].isna().value_counts()

filtered
False    13170
Name: count, dtype: int64

In [26]:
df_post['filtered'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13170 entries, 0 to 13169
Series name: filtered
Non-Null Count  Dtype 
--------------  ----- 
13170 non-null  object
dtypes: object(1)
memory usage: 103.0+ KB


In [45]:
import numpy as np

# Define a function to batch process the sentiment predictions
def batch_predict_sentiment(texts, model, batch_size=32):
    num_batches = len(texts) // batch_size + (len(texts) % batch_size != 0)
    sentiment_labels = []
    for i in range(num_batches):
        batch_texts = texts[i * batch_size: (i + 1) * batch_size]
        batch_predictions = [model(text)[0]['label'] for text in batch_texts]
        sentiment_labels.extend(batch_predictions)
    return sentiment_labels

# Define batch size
batch_size = 32

# Create an empty list to store sentiment labels
sentiment_labels = []

# Batch processing the sentiment predictions
for i in range(0, len(df_post['filtered']), batch_size):
    batch_texts = df_post['filtered'][i:i+batch_size].astype(str)
    batch_predictions = batch_predict_sentiment(batch_texts, model)
    sentiment_labels.extend(batch_predictions)

# Assign the sentiment labels to the DataFrame
df_post['Sentiment_lbl'] = sentiment_labels



In [38]:
df_post['filtered'].tail()

13165    Zydus Lifesciences, Poly Medicure among 10 sto...
13166    Technical breakout: SBI, Zydus Wellness, 5 oth...
13167    Stock Radar: Bullish crossover on RSI makes Zy...
13168    Zydus Wellness promoters' stake hike point to ...
13169    Voltas, Zydus Wellness, 5 other stocks surpass...
Name: filtered, dtype: object

In [47]:
df_post['Sentiment_lbl'].value_counts()

Sentiment_lbl
positive    5267
neutral     4607
negative    3296
Name: count, dtype: int64

In [49]:
import numpy as np

# Define a function to batch process the sentiment scores
def batch_predict_sentiment_scores(texts, model, batch_size=32):
    num_batches = len(texts) // batch_size + (len(texts) % batch_size != 0)
    sentiment_scores = []
    for i in range(num_batches):
        batch_texts = texts[i * batch_size: (i + 1) * batch_size]
        batch_predictions = [model(text)[0]['score'] for text in batch_texts]
        sentiment_scores.extend(batch_predictions)
    return sentiment_scores

# Define batch size
batch_size = 32

# Create an empty list to store sentiment scores
sentiment_scores = []

# Batch processing the sentiment scores
for i in range(0, len(df_post['filtered']), batch_size):
    batch_texts = df_post['filtered'][i:i+batch_size].astype(str)
    batch_scores = batch_predict_sentiment_scores(batch_texts, model)
    sentiment_scores.extend(batch_scores)

# Assign the sentiment scores to the DataFrame
df_post['Sentiment_score'] = sentiment_scores


In [52]:
df_post['Sentiment_scr'].value_counts()

Sentiment_scr
0.981129    13170
Name: count, dtype: int64

In [61]:
# Define a function to batch process the sentiment labels and scores
def batch_predict_sentiment(texts, model, batch_size=32):
    num_batches = len(texts) // batch_size + (len(texts) % batch_size != 0)
    sentiment_labels = []
    sentiment_scores = []
    for i in range(num_batches):
        batch_texts = texts[i * batch_size: (i + 1) * batch_size].tolist()  # Ensure texts are in list format
        batch_predictions = model(batch_texts)
        for prediction in batch_predictions:
            if isinstance(prediction, dict) and 'label' in prediction and 'score' in prediction:
                sentiment_labels.append(prediction['label'])
                sentiment_scores.append(prediction['score'])
    return sentiment_labels, sentiment_scores

# Define batch size
batch_size = 32

# Batch processing the sentiment labels and scores
sentiment_labels, sentiment_scores = batch_predict_sentiment(df_post['filtered'].astype(str), model, batch_size=batch_size)

# Assign the sentiment labels and scores to the DataFrame
df_post['Sent_lbl_new'] = sentiment_labels
df_post['Sent_scr_new'] = sentiment_scores


In [74]:
df_post.head(2)

,News,filtered,Date,Entity,Symbol,Open,Close,Perc_Change,Sentiment,Sentiment_lbl,Sentiment_scr,Sentiment_score,Sent_lbl_new,Sent_scr_new
0,"Technical Breakout Stocks: How to trade HEG, 3...","Technical Breakout Stocks: How to trade HEG, 3...",2023-12-27,3M India,3MINDIA,31127.0,31447.599609,1.029973,NaN,neutral,0.981129,0.999881,neutral,0.999881
1,"Aarti Industries, HDFC AMC among 11 stocks tha...","Aarti Industries, HDFC AMC among 11 stocks tha...",2020-07-01,Aarti Industries,AARTIIND,466.5,465.700012,-0.171487,NaN,neutral,0.981129,0.999881,neutral,0.999881


In [83]:

df_post.head(2)

,News,filtered,Date,Entity,Symbol,Open,Close,Perc_Change,Sent_lbl_new,Sent_scr_new
0,"Technical Breakout Stocks: How to trade HEG, 3...","Technical Breakout Stocks: How to trade HEG, 3...",2023-12-27,3M India,3MINDIA,31127.0,31447.599609,1.029973,neutral,0.999881
1,"Aarti Industries, HDFC AMC among 11 stocks tha...","Aarti Industries, HDFC AMC among 11 stocks tha...",2020-07-01,Aarti Industries,AARTIIND,466.5,465.700012,-0.171487,neutral,0.999881


In [84]:
df_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13170 entries, 0 to 13169
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   News          13170 non-null  object 
 1   filtered      13170 non-null  object 
 2   Date          13170 non-null  object 
 3   Entity        13170 non-null  object 
 4   Symbol        13170 non-null  object 
 5   Open          13141 non-null  float64
 6   Close         13141 non-null  float64
 7   Perc_Change   13141 non-null  float64
 8   Sent_lbl_new  13170 non-null  object 
 9   Sent_scr_new  13170 non-null  float64
dtypes: float64(4), object(6)
memory usage: 1.0+ MB


## Hypothesis Testing

In [65]:

from scipy.stats import f_oneway


# Group data by sentiment label
groups = {}
for label, group in df_post.groupby('Sent_lbl_new'):
    groups[label] = group['Perc_Change']

# Perform ANOVA
f_statistic, p_value = f_oneway(groups['positive'], groups['negative'], groups['neutral'])

# Print results
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis: There is a significant difference in stock price variation among different sentiment labels.")
else:
    print("Fail to reject null hypothesis: There is no significant difference in stock price variation among different sentiment labels.")


F-statistic: nan
P-value: nan
Fail to reject null hypothesis: There is no significant difference in stock price variation among different sentiment labels.


In [86]:
groups['neutral']

0        1.029973
1       -0.171487
2       -1.710155
3       -6.577778
4        1.665243
           ...   
13152    2.277437
13153    0.808630
13159   -0.523192
13162    4.015485
13166    2.273874
Name: Perc_Change, Length: 4607, dtype: float64

In [87]:
df_post.to_excel('D:\\MIT PGDAML\\Neural Network\\Project\\Filtered_News_ET - NextDay_FINAL.xlsx', index=True)